In [4]:
import requests, bs4
import mysql.connector
from lxml import html
from urllib.request import Request,urlopen
from urllib.parse import urlencode,quote_plus
from bs4 import BeautifulSoup
from xml.etree.ElementTree import parse
from datetime import datetime
from ServiceKey import public_serviceKey, mySQL_Connect

mydb = mySQL_Connect()
# 2021.09.18 자료 추출 완료
cursor = mydb.cursor(prepared=True)
citycode = '31010'

query = ("select distinct routeid from public_busroute_info where citycode = '{}'".format(citycode))

cursor.execute(query)

location_list = []

for i in cursor:
    location_list.append(i)

now = datetime.now()
nowDate = now.strftime('%Y%m%d')
    
cnt = 1
for i in location_list:
    routeId = i
    
    characters = "(',')"

    join_data = ''.join( x for x in routeId if x not in characters)
    print("조회날짜 : {} | cnt : {} | start {} public_busroute_info".format(nowDate,cnt,join_data))
    
    url = 'http://openapi.tago.go.kr/openapi/service/BusRouteInfoInqireService/getRouteAcctoThrghSttnList'
    queryParams = '?' + urlencode({
        quote_plus('serviceKey') : public_serviceKey(),
        quote_plus('cityCode') : citycode,
        quote_plus('pageNo') : '1',
        quote_plus('numOfRows') : '10000',
        quote_plus('routeId') : join_data
    })

    response = requests.get(url + queryParams).text.encode('utf-8')
    xmlobj = bs4.BeautifulSoup(response, 'lxml-xml')

    rows = xmlobj.findAll('item')
    resultCode = xmlobj.findAll('resultCode')
    if resultCode[0].text == '99':
        print("{} 번째부터 조회수 제한이 걸렸습니다.".format(join_data))
        mydb.close()
        break
    cnt += 1
    for j in rows:
        item = j
        lat = item.findAll('gpslati')
        lng = item.findAll('gpslong')
        nodeid = item.findAll('nodeid')
        nodenm = item.findAll('nodenm')
        nodeno = item.findAll('nodeno')
        nodeord = item.findAll('nodeord')
        routeid = item.findAll('routeid')
        updowncd = item.findAll('updowncd')
        try:
            if updowncd[0].text == '0':
                updowncd = '상행'
            elif updowncd[0].text == '1':
                updowncd = '하행'
        except IndexError:
            updowncd = None
        
        try:
            sql = "INSERT INTO public_busroute_detail (lat, lng, nodeid, nodenm, nodeno, nodeord, routeid, updowncd,citycode) VALUES (?,?,?,?,?,?,?,?,?)"
            val = (lat[0].text, lng[0].text, nodeid[0].text, nodenm[0].text, nodeno[0].text, nodeord[0].text, routeid[0].text, updowncd,citycode)
        except IndexError:
            sql = "INSERT INTO public_busroute_detail (lat, lng, nodeid, nodenm, nodeno, nodeord, routeid, updowncd,citycode) VALUES (?,?,?,?,?,?,?,?,?)"
            val = (lat[0].text, lng[0].text, nodeid[0].text, nodenm[0].text, None, nodeord[0].text, routeid[0].text, updowncd,citycode)
            
        cursor.execute(sql, val)
        mydb.commit()

조회날짜 : 20210918 | cnt : 1 | start GGB200000053 public_busroute_info
조회날짜 : 20210918 | cnt : 2 | start GGB200000104 public_busroute_info
조회날짜 : 20210918 | cnt : 3 | start GGB200000108 public_busroute_info
조회날짜 : 20210918 | cnt : 4 | start GGB200000109 public_busroute_info
조회날짜 : 20210918 | cnt : 5 | start GGB200000110 public_busroute_info
조회날짜 : 20210918 | cnt : 6 | start GGB200000112 public_busroute_info
조회날짜 : 20210918 | cnt : 7 | start GGB200000115 public_busroute_info
조회날짜 : 20210918 | cnt : 8 | start GGB200000119 public_busroute_info
조회날짜 : 20210918 | cnt : 9 | start GGB200000120 public_busroute_info
조회날짜 : 20210918 | cnt : 10 | start GGB200000145 public_busroute_info
조회날짜 : 20210918 | cnt : 11 | start GGB200000146 public_busroute_info
조회날짜 : 20210918 | cnt : 12 | start GGB200000150 public_busroute_info
조회날짜 : 20210918 | cnt : 13 | start GGB200000182 public_busroute_info
조회날짜 : 20210918 | cnt : 14 | start GGB200000205 public_busroute_info
조회날짜 : 20210918 | cnt : 15 | start GGB20000